In [88]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import time

# ----------------------------------
# 날짜 입력받기
#   -> 입력하지 않으면 오늘날짜 반환
#   -> 잘못된 형식이면 None 반환
# ----------------------------------
def date_input():

    # 날짜 입력받기
    date_str = input("날짜:(형식:20200105)")
    
    # 날짜를 입력하지 않으면 오늘 날짜 가져오기
    if date_str=="":
        date_str = datetime.now().strftime('%Y%m%d')
    
    # 날짜 형식 변환 (YYYYmmdd --> YYYY-mm-dd)
    try:
        date_str = datetime.strptime(date_str, '%Y%m%d').strftime('%Y-%m-%d')
    except:
        print("날짜 형식이 올바르지 않습니다.")
        date_str = None
    
    return date_str

date = date_input()

print(f'입력한 날짜 : {date}')


입력한 날짜 : 2025-08-10


In [89]:
   
# ----------------------------------
# 웹페이지 요청하여 응답객체 받기
# ----------------------------------

data_list = []

while True :
    
    url = f'https://finance.naver.com/news/mainnews.naver?date={date}&page={page}'

    response = requests.get(url)
    html = response.text

    # ----------------------------------
    # 응답받은 웹페이지 파싱하여 BeautifulSoup 객체 생성
    # ----------------------------------

    soup = BeautifulSoup(html, 'html.parser')


    # ----------------------------------
    # article 리스트 추출
    # ----------------------------------   

    articles = soup.select('.block1')

    # ----------------------------------
    # article 리스트에서 요소의 텍스트, 속성 추출
    # ----------------------------------   

    for article in articles :
        subject = article.select_one('.articleSubject > a').text

        # summary = article.select_one(".articleSummary").contents[0].text.strip()
        summary = article.select_one('.articleSummary').text.split(' ')[:-4]
        summary = ' '.join(summary).strip()

        # press = article.select_one(".articleSummary").contents[1].text.strip()
        press = article.select_one('.press').text

        # wdate = article.select_one(".articleSummary").contents[5].text.strip()
        wdate = article.select_one('.wdate').text

        link = article.select_one('.articleSubject > a').attrs['href']

        # article_id = link.split('=')[1].split('&')[0]
        # office_id = link.split('office_id=')[1].split('&')[0]
        # link = f'https://n.news.naver.com/mnews/article/{office_id}/{article_id}'

        data_list.append({'subject' : subject,
                        'summary' : summary,
                        'press' : press,
                        'wdate' : wdate,
                        'link' : link})

    # ----------------------------------
    # 페이지 이동
    # ----------------------------------   

    if soup.select_one('.pgRR') :
        print(page)
        page += 1
    else :
        break

    time.sleep(3)



In [90]:
# ---------------------------
# 데이터프레임 생성
# ---------------------------

df = pd.DataFrame(data_list)
df

,subject,summary,press,wdate,link
0,다단계 의심받던 시골 회사의 반전…'800억 부자' 된 아빠와 딸 [윤현주의 主食이...,백문불여일견(百聞不如一見). 백 번 듣는 것보다 한 번 보는 게 낫다는 말이다. 가...,한국경제,2025-08-10 07:01:13,/news/news_read.naver?article_id=0005168753&of...
1,"경기 불안한데 상하이 지수 최고치 경신, 왜?…“희토류 패권 쥔 중국, 투자 매력 ...",7일 상하이종합지수 3639.67…연초 대비 27% 상승 중국 경기 둔화에도 최고점...,헤럴드경제,2025-08-10 07:00:20,/news/news_read.naver?article_id=0002512132&of...
2,달러 약세 흐름 지속…미 CPI 대기하며 방향성 탐색[주간외환전망],이번 주 외환 시장은 미국 7월 소비자물가지수(CPI) 발표를 주시하며 미국 연방준...,이데일리,2025-08-10 07:00:17,/news/news_read.naver?article_id=0006086277&of...
3,관세 리스크에 M&A 지형도 재편…산업별 ‘엇갈린 성적표’ [투자360],의류·여행업 침체…미주 중심 재편 하반기 운송·물류 불확실성 지속 [헤럴드경제=노아...,헤럴드경제,2025-08-10 07:00:13,/news/news_read.naver?article_id=0002512130&of...
4,잭슨홀 테이블에 올라갈 CPI…경계심 고조[뉴욕증시-주간전망],진정호 연합인포맥스 특파원 = 이번 주 뉴욕증시는 물가지표가 향방을 좌우하는 관건이...,연합뉴스,2025-08-10 07:00:00,/news/news_read.naver?article_id=0015557407&of...
5,"새내기주는 '따블', 대어는 '쑥'…다시 부는 공모주 열기",최근 국내 주식시장에 입성한 코스피와 코스닥 새내기주에 연일 훈풍이 분다. 이번주(...,머니투데이,2025-08-10 07:00:00,/news/news_read.naver?article_id=0005233692&of...
6,"""코스피 더 간다?""…7월 차익 실현한 개미들, 8월 다시 '컴백'",코스피 랠리 후 차익실현에 나섰던 개미(개인투자자)들이 다시 돌아왔다. 10일 한국...,뉴스1,2025-08-10 07:00:00,/news/news_read.naver?article_id=0008419893&of...
7,하루 만에 168% 폭등…개미들 몰린 '이곳' 뭐길래,새내기주가 증시 상장 첫날 줄줄이 급등하면서 단기 투자처로 떠오르고 있다. 9일 한...,한국경제,2025-08-10 06:48:16,/news/news_read.naver?article_id=0005168752&of...
8,"관세 충격에도 선방 평가 속…토요타, 실적 발표 하루 만에 주가 반등",일본 토요타자동차가 미국발(發) 관세 충격에도 올해 회계연도 실적 전망 하향 폭을 ...,이데일리,2025-08-10 06:30:14,/news/news_read.naver?article_id=0006086275&of...
9,"“美 조선업 부활” 외친 트럼프…실제 수혜는 HD현대중공업·한화오션, 이유는? [투...",미국 내 조선소 노후화·인력 공백 심화 중국 견제 위해 ‘美 조선업 재건’ 판단 ‘...,헤럴드경제,2025-08-10 06:30:13,/news/news_read.naver?article_id=0002512129&of...


In [91]:
# ------------
# csv 파일로 저장
# ------------

df.to_csv(f'data/네이버증권뉴스({date}).csv')

In [92]:
# ------------
# excel 파일로 저장
# ------------

df.to_excel(f'data/네이버증권뉴스({date}).xlsx')